# ecg

```ecg``` is a library that works with electrocardiogram signal. It allows easily load and process ecg signal and learn any model. The library is based on [Dataset]() and supports its whole functionality. So you can define your oun pipeline, write custon preprocess functions or use built-in ones and handle with datasets even if it does not fit into memory.

```ecg``` has two modules: [```batch```]() and [```models```](). 

In ```batch``` we gather everything you may need to process ecg signal:
* load and save signal in a number of formats
* resample, crop or flip signal
* filter signal
* apply complex transformations like fft or wavelets
* ...

In ```models``` we provide several models that should inspire you to start you ouw research. Provided models are created to learn the most important problems in ecg:
* how to recognize specific features of ecg like R-peaks, P-wave, T-wave
* how to recignize dengerous deseases from ecg, for example - atrial fibrillation.

# Basic usage

Here is an example of pipeline that loads ecg signals, make some preprocessing and learns model over 50 epochs.
```
model_train_pipeline = (ds.Pipeline()
                        .load(fmt="wfdb", components=["signal", "meta"])
                        .load(src="/notebooks/data/REFERENCE.csv", fmt="csv", components="target")
                        .drop_labels(["~"])
                        .replace_labels({"N": "NO", "O": "NO"})
                        .random_resample_signals("normal", loc=300, scale=10)
                        .drop_short_signals(3000)
                        .segment_signals(3000, 1000
                        )
                        .binarize_labels()
                        .train_on_batch('my_ecg_model', metrics=f1_score, average='macro')
                        .run(batch_size=300, shuffle=True, n_epochs=50, prefetch=0))
```

# Batch

## ecg_batch
This is a base class that defines how to store ecg data and lists all the actions that can be applied to ecg regardless of models. Class ```ecg_base``` has following attributes:
* ```signal``` stores ecg signal as numpy array
* ```annotation```, this can be any array that annotate specific points of ecg signal, e.g. R peaks
* ```meta``` contains any parameters of ecg signal, e.g. signal frequency
* ```target``` contains label assigned to ecg
* ```unique_labels``` is just a list of all possible target labels.

## Actions of ecg_batch

### ```load(self, src=None, fmt=None, components=None, *args, **kwargs)```

### ```drop_labels(self, drop_list)```

### ```keep_labels(self, keep_list)```

### ```replace_labels(self, replace_dict)```

### ```binarize_labels```

### ```drop_short_signals(self, min_length, axis=-1)```

### ```segment_signals(self, index, length, step, pad_value=0)```

### ```random_segment_signals(self, index, length, n_segments, pad_value=0)```

### ```convolve_signals(self, kernel, padding_mode="edge", axis=-1, **kwargs)```

### ```band_pass_signals(self, index, low=None, high=None, axis=-1)```


## ecg_batch_tools

Contains helpful functions that typically are called from some action of the ecg_batch for each signal separately. 


## kernels

Contains kernel generation functions.

## model_ecg_batch

Contains actions that initialize models and common action for training, testing and predicting models.

### ```fft_pretrained(pipeline, config=None)```
Load pretrained [FFT]() model.

### ```dirichlet_pretrained(pipeline, config=None)```
Load pretrained [Dirichlet]() model.

### ```train_on_batch(self, model_name, *args, **kwargs)```

### ```test_on_batch(self, model_name, *args, **kwargs)```

### ```predict_on_batch(self, model_name, *args, **kwargs)```

## utils

Miscellaneous ECG Batch utils